Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [118]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf
from pandas import DataFrame
from sklearn.linear_model import LinearRegression

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (20000, 28, 28) (20000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (20000, 784) (20000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [48]:
hidden_size = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Variables.
  weights_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))

  biases_hidden = tf.Variable(tf.zeros([hidden_size]))

  weights = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  def inference(data):
    hidden_relu = tf.nn.relu(tf.matmul(data, weights_hidden) + biases_hidden)
    logits = tf.matmul(hidden_relu, weights) + biases
    return logits

  train_logits = inference(tf_train_dataset)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(train_logits)
  valid_prediction = tf.nn.softmax(inference(tf_valid_dataset))
  test_prediction = tf.nn.softmax(inference(tf_test_dataset))

  cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(train_logits, tf_train_labels))
  l2_loss = tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights)
  
  loss = cross_entropy + l2_loss * 0.001
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [49]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 628.956
Minibatch accuracy: 6.2%
Validation accuracy: 30.1%
Minibatch loss at step 500 : 190.001
Minibatch accuracy: 90.6%
Validation accuracy: 80.3%
Minibatch loss at step 1000 : 118.025
Minibatch accuracy: 91.4%
Validation accuracy: 81.3%
Minibatch loss at step 1500 : 69.3027
Minibatch accuracy: 97.7%
Validation accuracy: 83.2%
Minibatch loss at step 2000 : 42.0285
Minibatch accuracy: 100.0%
Validation accuracy: 83.5%
Minibatch loss at step 2500 : 25.533
Minibatch accuracy: 99.2%
Validation accuracy: 84.0%
Minibatch loss at step 3000 : 15.5622
Minibatch accuracy: 100.0%
Validation accuracy: 84.8%
Test accuracy: 91.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [54]:
hidden_size = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Variables.
  weights_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))

  biases_hidden = tf.Variable(tf.zeros([hidden_size]))

  weights = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  def inference(data, keep_proba=1):
    dropout = tf.nn.dropout(tf.matmul(data, weights_hidden) + biases_hidden, keep_proba)
    hidden_relu = tf.nn.relu(dropout)
    logits = tf.matmul(hidden_relu, weights) + biases
    return logits

  train_logits = inference(tf_train_dataset, keep_proba=0.5)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(train_logits)
  valid_prediction = tf.nn.softmax(inference(tf_valid_dataset))
  test_prediction = tf.nn.softmax(inference(tf_test_dataset))

  cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(train_logits, tf_train_labels))
  l2_loss = tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights)
  
  loss = cross_entropy + l2_loss * 0.001
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [55]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 800.766
Minibatch accuracy: 8.6%
Validation accuracy: 27.4%
Minibatch loss at step 500 : 201.008
Minibatch accuracy: 75.0%
Validation accuracy: 80.2%
Minibatch loss at step 1000 : 118.267
Minibatch accuracy: 72.7%
Validation accuracy: 79.6%
Minibatch loss at step 1500 : 69.5161
Minibatch accuracy: 88.3%
Validation accuracy: 82.6%
Minibatch loss at step 2000 : 42.2701
Minibatch accuracy: 84.4%
Validation accuracy: 83.5%
Minibatch loss at step 2500 : 25.662
Minibatch accuracy: 93.8%
Validation accuracy: 84.6%
Minibatch loss at step 3000 : 15.7503
Minibatch accuracy: 90.6%
Validation accuracy: 85.3%
Test accuracy: 91.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [143]:
df = DataFrame(columns=('hidden_size', 
                        'batch_size', 
                        'layer_1_keep_proba', 
                        'layer_2_keep_proba',
                        'weights_1_penalty',
                        'weights_2_penalty',
                        'weights_penalty',
                        'step_size',
                        'num_steps',
                        'test_accuracy',
                        'slope'))

In [244]:
df['step_size_decay'] = [1] * 28

,hidden_size,batch_size,layer_1_keep_proba,layer_2_keep_proba,weights_1_penalty,weights_2_penalty,weights_penalty,step_size,num_steps,test_accuracy,slope,step_size_decay
0,1024,128,0.90,0.50,0.050,0.005,0.0100,0.0005,11,41.839351,2.297000,1
0,1024,128,0.90,0.50,0.050,0.005,0.0100,0.0005,3001,89.430677,0.000330,1
0,1024,128,0.90,0.50,0.050,0.005,0.0010,0.0005,3001,89.708396,0.000015,1
0,1024,128,0.90,0.50,0.010,0.005,0.0010,0.0005,3001,89.366588,0.000270,1
0,1024,128,0.50,0.50,0.010,0.005,0.0010,0.0005,3001,89.158300,0.000180,1
0,1024,128,0.50,0.50,0.005,0.005,0.0010,0.0005,5001,89.500107,0.000015,1
0,1024,128,1.00,0.50,0.050,0.005,0.0010,0.0005,5001,89.345225,0.000150,1
0,512,128,1.00,0.50,0.050,0.005,0.0010,0.0005,5001,86.349071,-0.000065,1
0,1024,128,0.90,0.50,0.100,0.005,0.0010,0.0005,5001,89.532151,0.000090,1
0,1024,128,0.90,0.50,0.100,0.005,0.0010,0.0002,5001,89.334544,0.000200,1


In [356]:
hidden_size = 1024
batch_size = 128

# dropout probabilities. Lower numbers mean more dropout and therefore more regularization
layer_1_keep_proba = 0.5
layer_2_keep_proba = 0.5
#L2 penalty for weights. Higher numbers mean more penalty, and therefore more regularization
weights_1_penalty = 0.025
weights_2_penalty = 0.01
weights_penalty = 0.02
# Step size
starting_step_size = 0.005
step_size = starting_step_size
step_size_decay = 0.9
MOVING_AVERAGE_DECAY = 0.9995

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  biases1 = tf.Variable(tf.zeros([hidden_size]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_size, hidden_size]))
  biases2 = tf.Variable(tf.zeros([hidden_size]))

  weights = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  # Track the moving averages of all trainable variables.
  variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
  variable_averages_op = variable_averages.apply(tf.trainable_variables())

  def getVar(v, averaged):
        if averaged:
            return variable_averages.average(v)
        else:
            return v
  
  # Training computation.
  def inference(data, dropout=False, averaged=False):
    # Dropout for training, no dropout for inference
    if not dropout:
        l1_dropout = 1
        l2_dropout = 1
    else:
        l1_dropout = layer_1_keep_proba
        l2_dropout = layer_2_keep_proba
        
    # Get the current version for training or the averaged version for 
    # evaluation
    w1 = getVar(weights1, averaged)
    b1 = getVar(biases1, averaged)
    w2 = getVar(weights2, averaged)
    b2 = getVar(biases2, averaged)
    w = getVar(weights, averaged)
    b = getVar(biases, averaged)
    
    # actual network archetecture
    relu1 = tf.nn.relu(tf.matmul(data, w1) + b1)
    dropout1 = tf.nn.dropout(relu1, l1_dropout)
    relu2 = tf.nn.relu(tf.matmul(dropout1, w2) + b2)
    dropout2 = tf.nn.dropout(relu2, l2_dropout)
    logits = tf.matmul(dropout2, w) + b
    return logits

  train_logits = inference(tf_train_dataset, dropout=True)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(train_logits)
  valid_prediction = tf.nn.softmax(inference(tf_valid_dataset, averaged=True))
  test_prediction = tf.nn.softmax(inference(tf_test_dataset, averaged=True))

  cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(train_logits, tf_train_labels))

  l2_loss = (weights_1_penalty * tf.nn.l2_loss(weights1) 
             + weights_2_penalty * tf.nn.l2_loss(weights2)
             + weights_penalty * tf.nn.l2_loss(weights))
  
  loss = cross_entropy + l2_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(step_size).minimize(loss)

In [357]:
num_steps = 5001
validation_accuracies = []
step_num = []

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions, _ = session.run(
      [optimizer, loss, train_prediction, variable_averages_op], feed_dict=feed_dict)
    
    if (step % 200 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      valid_accuracy = accuracy(
        valid_prediction.eval(), valid_labels)
      print "Validation accuracy: %.1f%%" % valid_accuracy
      validation_accuracies.append(valid_accuracy)
      step_num.append(step)
      step_size *= step_size_decay
    
  test_accuracy = accuracy(test_prediction.eval(), test_labels)
  print "Test accuracy: %.1f%%" % test_accuracy
  run_info = dict()
  run_info['hidden_size'] = hidden_size 
  run_info['batch_size'] = batch_size
  run_info['layer_1_keep_proba'] = layer_1_keep_proba
  run_info['layer_2_keep_proba'] = layer_2_keep_proba
  run_info['weights_1_penalty'] = weights_1_penalty
  run_info['weights_2_penalty'] = weights_2_penalty
  run_info['weights_penalty'] = weights_penalty
  run_info['step_size'] = starting_step_size
  run_info['num_steps'] = num_steps
  run_info['test_accuracy'] = test_accuracy
  run_info['step_size_decay'] = step_size_decay
  
  X = np.array(step_num[-5:]).reshape((-1, 1))
  y = np.array(validation_accuracies[-5:])
  lr = LinearRegression()
  lr.fit(X, y)
  # Measures how much the error is continuing to decrease at the end of the run.
  # values above 0 suggest longer training time.  
  run_info['slope'] = lr.coef_[0]
  df = df.append([run_info])

Initialized
Minibatch loss at step 0 : 26360.6
Minibatch accuracy: 12.5%
Validation accuracy: 8.7%
Minibatch loss at step 200 : 12620.4
Minibatch accuracy: 68.8%
Validation accuracy: 16.1%
Minibatch loss at step 400 : 11394.7
Minibatch accuracy: 71.9%
Validation accuracy: 26.3%
Minibatch loss at step 600 : 10810.0
Minibatch accuracy: 70.3%
Validation accuracy: 36.8%
Minibatch loss at step 800 : 10335.6
Minibatch accuracy: 71.9%
Validation accuracy: 51.3%
Minibatch loss at step 1000 : 9894.1
Minibatch accuracy: 71.1%
Validation accuracy: 62.3%
Minibatch loss at step 1200 : 9442.6
Minibatch accuracy: 67.2%
Validation accuracy: 68.9%
Minibatch loss at step 1400 : 9007.43
Minibatch accuracy: 73.4%
Validation accuracy: 72.7%
Minibatch loss at step 1600 : 8660.94
Minibatch accuracy: 75.0%
Validation accuracy: 75.1%
Minibatch loss at step 1800 : 8326.05
Minibatch accuracy: 70.3%
Validation accuracy: 76.8%
Minibatch loss at step 2000 : 8019.69
Minibatch accuracy: 70.3%
Validation accuracy: 78.

In [304]:
df.sort_values('test_accuracy')

,hidden_size,batch_size,layer_1_keep_proba,layer_2_keep_proba,weights_1_penalty,weights_2_penalty,weights_penalty,step_size,num_steps,test_accuracy,slope,step_size_decay
0,1024,128,0.90,0.50,0.050,0.005,0.0100,0.000500,11,41.839351,2.297000,1.0
0,1024,128,0.50,0.50,0.010,0.010,0.0010,0.000046,10001,80.954924,-0.000840,NaN
0,512,128,1.00,0.50,0.050,0.005,0.0010,0.000500,5001,86.349071,-0.000065,1.0
0,1024,128,0.90,0.50,0.050,0.001,0.0005,0.000200,4001,88.784448,0.000230,1.0
0,1024,128,0.90,0.50,0.050,0.005,0.0003,0.000200,4001,88.987396,0.000125,1.0
0,1024,128,0.90,0.75,0.050,0.005,0.0010,0.000200,6001,89.035463,0.000170,1.0
0,1024,128,0.90,0.50,0.050,0.010,0.0005,0.000200,4001,89.099551,0.000265,1.0
0,1024,128,0.90,0.50,0.050,0.005,0.0030,0.000200,4001,89.131596,0.000040,1.0
0,1024,128,0.50,0.50,0.010,0.005,0.0010,0.000500,3001,89.158300,0.000180,1.0
0,1024,128,0.50,0.50,0.050,0.010,0.0010,0.000067,8001,89.190344,-0.000270,NaN
